In [ ]:
# default_exp core

# The foundations

> Let's try to replicate the standard usage of MNL frameworks

We will try to implement a basic MNL package to compare against biogeme/others... let's start from this [blog post](https://aaronkub.com/2020/02/12/logistic-regression-with-pytorch.html)

In [ ]:
#export
import random

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from fastprogress import progress_bar, master_bar
from fastcore.all import *

grab some data

In [ ]:
data = pd.read_csv("./data/Iris.csv").drop("Id", axis=1)

X_numpy = data.drop("Species", axis=1).values

target_map = {
    val: index for index, val in enumerate(data.Species.unique())
}
y_numpy = data.Species.map(target_map).values

X = torch.tensor(X_numpy, dtype=torch.float32)
y = torch.tensor(y_numpy)

In [ ]:
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
target_map

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [ ]:
#export
def prepare_data(data, x_cols=None, target_col=None, av_cols=None):
    "This is far from optimal, as we shuould be reading values lazily"
    target_col = ifnone(target_col, list(data.columns)[-1])
    x_cols = [col for col in ifnone(x_cols, list(data.columns)) if col!=target_col]
    X_numpy = data.loc[:, x_cols].values
    target_map = {
        val: index for index, val in enumerate(data.loc[:,target_col].unique())
    }
    print(target_map)
    y_numpy = data.loc[:,target_col].map(target_map).values
    
    X = torch.tensor(X_numpy, dtype=torch.float32)
    y = torch.tensor(y_numpy)
    if av_cols is None:
        return X, y, torch.ones(len(X), len(target_map))
    else:
        return X, y, torch.tensor(data.loc[:, av_cols].values, dtype=torch.float32)

In [ ]:
X, y, available_choices = prepare_data(data)

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


In [ ]:
ds = TensorDataset(X, y, available_choices)

In [ ]:
X[0:5], y[0:5], available_choices[0:5]

(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
         [4.9000, 3.0000, 1.4000, 0.2000],
         [4.7000, 3.2000, 1.3000, 0.2000],
         [4.6000, 3.1000, 1.5000, 0.2000],
         [5.0000, 3.6000, 1.4000, 0.2000]]),
 tensor([0, 0, 0, 0, 0]),
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]))

In [ ]:
len(X), len(y)

(150, 150)

## Train/Test split

In [ ]:
random.shuffle(range_of(X))

In [ ]:
#export
def train_valid_split(*tensors, pct=0.2, shuffle=True):
    assert len(tensors[0]) == len(tensors[1]), "X and y don't have the same number of elements"
    indices = range_of(tensors[0])
    if shuffle:
        random.shuffle(indices)
    n = len(tensors[0])
    n_train = int(n * (1-0.2))
    train_tensors = tuple(t[indices[:n_train]] for t in tensors)
    valid_tensors = tuple(t[indices[n_train:]] for t in tensors)
    return train_tensors, valid_tensors

In [ ]:
_X = L([0,1,2,3,4,5,6,7,8,9])
_y = L('a,b,c,d,e,f,g,h,i,j'.split(','))

test_eq(train_valid_split(_X,_y, shuffle=False)[0][0], [0,1,2,3,4,5,6,7])
test_eq(train_valid_split(_X,_y, shuffle=False)[1][1], ['i','j'])

## Model

In [ ]:
#export
class Availability(nn.Module):
    
    def __init__(self, n_choices, inf=1e6):
        super().__init__()
        store_attr()
        
    def forward(self, x, av):
        return x + (av - 1)*self.inf 

In [ ]:
av = Availability(3)

In [ ]:
F.softmax(av(torch.tensor([1,2,3]), torch.tensor([1,1,0])))

tensor([0.2689, 0.7311, 0.0000])

In [ ]:
#export
class LinearMNL(nn.Module):
    
    def __init__(self, in_dim=4, out_dim=5, bias=False):
        super().__init__()
        store_attr()
        self.linear = nn.Linear(in_dim, out_dim, bias=bias)
        self.availability = Availability(out_dim)
        
    def forward(self, x, av):
        return self.availability(self.linear(x), av)

In [ ]:
model = LinearMNL(4,3)
x = torch.rand(10,4)

model(x, torch.tensor([1,1,1])).shape

torch.Size([10, 3])

In [ ]:
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
#export
class DataLoaders:
    """
    A class to store dataloaders (train/valid/test....)"""
    def __init__(self, train_dl, valid_dl=None):
        store_attr()
        
    def one_batch(self, dl=None):
        dl = ifnone(dl, self.train_dl)
        return next(iter(dl))
    
    @delegates(DataLoader, but='batch_size')
    @classmethod
    def from_datasets(cls, train_ds, valid_ds=None, batch_size=1, **kwargs):
        train_dl = DataLoader(train_ds, batch_size=batch_size, **kwargs)
        if valid_ds is not None:
            valid_dl = DataLoader(valid_ds, batch_size=2*batch_size, **kwargs)
        else:
            valid_dl = None
        return cls(train_dl, valid_dl)
    
    @delegates(DataLoader, but='batch_size')
    @classmethod
    def from_Xy(cls, X, y, available_choices, pct=None, batch_size=1, **kwargs):
        if pct is not None:
            train_tensors, valid_tensors = train_valid_split(X, y, available_choices, pct=pct)
        else:
            train_tensors = X, y, available_choices
            valid_tensors = None
        train_ds = TensorDataset(*train_tensors)
        if valid_tensors is not None:
            valid_ds = TensorDataset(*train_tensors)
        else:
            valid_ds = None
        return cls.from_datasets(train_ds, valid_ds, batch_size)

In [ ]:
dls = DataLoaders.from_Xy(X, y, available_choices, pct=0.2, batch_size=8)

In [ ]:
#export
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=2, min_delta=0.001):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [ ]:
#export
class Learner:
    "A wrapper around dls, model and optimizer"
    def __init__(self, dls, model, loss_func=torch.nn.CrossEntropyLoss()):
        store_attr()
        

    def train_one_epoch(self):
        accum_loss = 0.
        for batch, (x, y, av) in enumerate(self.dls.train_dl):
            pred = self.model(x, av)  # 1
            loss = self.loss_func(pred, y)

            #backprop
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            accum_loss += loss.item()
        return accum_loss
    
    def validate(self, dl=None):
        dl = ifnone(dl, self.dls.valid_dl)
        if (dl is None):
            dl = self.dls.train_dl
        val_loss, accu = 0, 0
        with torch.no_grad():
            for batch, (x, y, av) in enumerate(dl):
                pred = self.model(x, av)
                val_loss += self.loss_func(pred, y).item()
                accu += (pred.argmax(1) == y).type(torch.float).sum().item()
        return val_loss, accu / len(dl.dataset)
    
    def fit(self, n_epochs=10, lr=0.01, wd=0.01):
        
        self.optimizer = torch.optim.SGD(
            self.model.parameters(), 
            lr=lr, 
            weight_decay=wd
        )
        
        early_stopping = EarlyStopping()
        
        scheduler = ReduceLROnPlateau(self.optimizer, 'min', patience=2, threshold=0.01, verbose=True)
        
        for epoch in progress_bar(range_of(n_epochs), leave=False):
            loss = self.train_one_epoch()
            val_loss, accuracy = self.validate()
            scheduler.step(val_loss)
            early_stopping(val_loss)
            if early_stopping.early_stop:
                break
            print(f'epoch = {epoch:3.0f}, train_loss = {loss:.3f}, val_loss = {val_loss:.3f}, accuracy = {accuracy:.2f}')

In [ ]:
model = LinearMNL(4,3)
learn = Learner(dls, model, loss_func)

In [ ]:
learn.fit(100, lr=0.1)

epoch =   0, train_loss = 18.694, val_loss = 6.437, accuracy = 0.66
epoch =   1, train_loss = 10.229, val_loss = 5.052, accuracy = 0.66
epoch =   2, train_loss = 8.974, val_loss = 4.628, accuracy = 0.66
epoch =   3, train_loss = 8.383, val_loss = 4.289, accuracy = 0.66
epoch =   4, train_loss = 7.884, val_loss = 3.944, accuracy = 0.66
epoch =   5, train_loss = 7.401, val_loss = 3.618, accuracy = 0.67
epoch =   6, train_loss = 6.941, val_loss = 3.328, accuracy = 0.68
epoch =   7, train_loss = 6.527, val_loss = 3.079, accuracy = 0.77
epoch =   8, train_loss = 6.179, val_loss = 2.869, accuracy = 0.80
epoch =   9, train_loss = 5.905, val_loss = 2.694, accuracy = 0.84
epoch =  10, train_loss = 5.693, val_loss = 2.546, accuracy = 0.88
epoch =  11, train_loss = 5.524, val_loss = 2.422, accuracy = 0.90
epoch =  12, train_loss = 5.383, val_loss = 2.316, accuracy = 0.92
epoch =  13, train_loss = 5.258, val_loss = 2.226, accuracy = 0.94
epoch =  14, train_loss = 5.142, val_loss = 2.148, accuracy 

## Export

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
Converted swissmetro.ipynb.
